# Generic setup stuff

In [1]:
# Set up and load data
# Includes
import sys
import os


# Setup paths containing utility
curr_folder = os.getcwd()
sys.path.insert(0, os.path.join(curr_folder,'../app'))

# Load the data
from utils import load_SQuAD_train
arts = load_SQuAD_train()


In [2]:
# Set up and test AllenNLP
from allennlp.predictors import Predictor
predictor = Predictor.from_path("/home/davestanley/src/allennlp/ner-model-2018.12.18.tar.gz")
# results = predictor.predict(sentence="Did Uriah honestly think he could beat The Legend of Zelda in under three hours?")
# for word, tag in zip(results["words"], results["tags"]):
#     print(f"{word}\t{tag}")




01/29/2019 11:05:34 - INFO - allennlp.models.archival -   loading archive file /home/davestanley/src/allennlp/ner-model-2018.12.18.tar.gz
01/29/2019 11:05:34 - INFO - allennlp.models.archival -   extracting archive file /home/davestanley/src/allennlp/ner-model-2018.12.18.tar.gz to temp dir /tmp/tmpfkj_gkmf
01/29/2019 11:05:39 - INFO - allennlp.common.params -   type = default
01/29/2019 11:05:39 - INFO - allennlp.data.vocabulary -   Loading token dictionary from /tmp/tmpfkj_gkmf/vocabulary.
01/29/2019 11:05:39 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.models.model.Model'> from params {'encoder': {'bidirectional': True, 'dropout': 0.5, 'hidden_size': 200, 'input_size': 1202, 'num_layers': 2, 'type': 'lstm'}, 'include_start_end_transitions': False, 'dropout': 0.5, 'type': 'crf_tagger', 'label_encoding': 'BIOUL', 'regularizer': [['scalar_parameters', {'alpha': 0.1, 'type': 'l2'}]], 'text_field_embedder': {'tokens': {'embedding_dim': 50, 'trainable': Tru

In [3]:
art = arts[105]

In [5]:
print(art['title'])

Pitch_(music)


In [7]:
# Choose a paragraph
paragraph = art['paragraphs'][0]['context']
print(paragraph)

Pitch is an auditory sensation in which a listener assigns musical tones to relative positions on a musical scale based primarily on their perception of the frequency of vibration. Pitch is closely related to frequency, but the two are not equivalent. Frequency is an objective, scientific attribute that can be measured. Pitch is each person's subjective perception of a sound, which cannot be directly measured. However, this does not necessarily mean that most people won't agree on which notes are higher and lower.


In [17]:
# Candidate blanks
[a['text'] for qa in art['paragraphs'][0]['qas'] for a in qa['answers']]

['auditory', 'frequency', 'Pitch']

# Parse the paragraph

In [8]:
# Import modules specific to this code
import random
import nltk
from nltk.tokenize import sent_tokenize

# Set up other global variables
debug_mode = True

In [9]:
# Split into sentences
sentences = sent_tokenize(paragraph)
Nsent = len(sentences)
if debug_mode:
    print("Nsentences=" + str(Nsent))

Nsentences=5


In [10]:
# Run it on my test sentence
results = predictor.predict(sentence=paragraph)
for word, tag in zip(results["words"], results["tags"]):
    if debug_mode:
        print(f"{word}\t{tag}")
    
words = results['words']
tags = results['tags']

Pitch	O
is	O
an	O
auditory	O
sensation	O
in	O
which	O
a	O
listener	O
assigns	O
musical	O
tones	O
to	O
relative	O
positions	O
on	O
a	O
musical	O
scale	O
based	O
primarily	O
on	O
their	O
perception	O
of	O
the	O
frequency	O
of	O
vibration	O
.	O
Pitch	O
is	O
closely	O
related	O
to	O
frequency	O
,	O
but	O
the	O
two	O
are	O
not	O
equivalent	O
.	O
Frequency	O
is	O
an	O
objective	O
,	O
scientific	O
attribute	O
that	O
can	O
be	O
measured	O
.	O
Pitch	O
is	O
each	O
person	O
's	O
subjective	O
perception	O
of	O
a	O
sound	O
,	O
which	O
can	O
not	O
be	O
directly	O
measured	O
.	O
However	O
,	O
this	O
does	O
not	O
necessarily	O
mean	O
that	O
most	O
people	O
wo	O
n't	O
agree	O
on	O
which	O
notes	O
are	O
higher	O
and	O
lower	O
.	O


In [11]:

# Get all named entity tags in the tags list
def find_inds_of_NE(tags):
    # Find list of named entitites
    l = [i for i, t in enumerate(tags) if not t == 'O']
    return(l)

l_NE = find_inds_of_NE(tags)
if debug_mode:
    print("Indices of named entities:" + str(l_NE))


Indices of named entities:[]


In [175]:
def join_punctuation(seq, characters='.,;?!'):
    # For joining lists of words together with correct
    # punctuation
    characters = set(characters)
    seq = iter(seq)
    current = next(seq)

    for nxt in seq:
        if nxt in characters:
            current += nxt
        else:
            yield current
            current = nxt

    yield current


# Choose one at random
import random
ind = random.choice(l_NE)

# Back up blanked out word and word type
removed_word = words[ind]
removed_word_tag = words[ind]

# Replace chosen word with blank
words_new = words.copy()
blank_token = '____'
words_new[ind] = blank_token

# Rebuild the sentence with appropriate punctuation
#paragraph_new = ' '.join(words_new).replace(" ,", ",").replace(" .", ".")
paragraph_new = ' '.join(join_punctuation(words_new))

# Print this sentence along with the previous sentence together
if debug_mode: print(paragraph_new)
if debug_mode: print("Answer: " + removed_word)



The term was created in 1920 by Hans Winkler, professor of botany at the ____ of Hamburg, Germany. The Oxford Dictionary suggests the name to be a blend of the words gene and chromosome. However, see omics for a more thorough discussion. A few related -ome words already existed — such as biome, rhizome, forming a vocabulary into which genome fits systematically.
Answer: University


Finally, figure out which sentence contains the blank and only present it and the previous two
===

In [312]:
# First, figure out the index of the sentence containing the blank
sentences_new = sent_tokenize(paragraph_new)


curr_word = 0
i=0 
for sent in sentences_new:
    i=i+1
    curr_word = curr_word + len(sent.split())
    if ind < curr_word:
        break

ind_sentence_containing_blank = i-1
if debug_mode: print(ind_sentence_containing_blank)


0


In [313]:
# if debug_mode:
#     # This method searches for the token directly - it is slower, but guaranteed to work)
#     # Never mind, punctuation messes this method up. No time to fix
#     sentences_new = sent_tokenize(paragraph_new)


#     i=0
#     for sent in sentences_new:
#         if blank_token in sent.split():
#             break
#         i=i+1
#     ind_sentence_containing_blank2 = i
#     if debug_mode: print(ind_sentence_containing_blank2)
#     if not ind_sentence_containing_blank2 == ind_sentence_containing_blank: print('Error occurred')



# Choose a subset of ~2 senteces preceeding the one containing the blank to display

In [252]:
def get_two_preceeding_sentences(sentences,ind):
    debug_mode = False
    sentences_subset = []

    # Store current and previous sentences
    sentences_subset.append(sentences[ind-2] if ind >= 2 else str(''))
    sentences_subset.append(sentences[ind-1] if ind >= 1 else str(''))
    sentences_subset.append(sentences[ind])

    if debug_mode:
        print("Current sentence number=" + str(ind))
        print("Sentence i-2 = " + sentences_subset[0])
        print("Sentence i-1 = " + sentences_subset[1])
        print("Sentence i = " + sentences_subset[2])
        
    return sentences_subset


sentences_subset = get_two_preceeding_sentences(sentences_new,ind_sentence_containing_blank)
paragraph_subset = ' '.join(join_punctuation([sent for sent in sentences_subset if sent]))  # Joins only if sentence is non-empty

print(paragraph_subset)

The term was created in 1920 by Hans Winkler, professor of botany at the University of Hamburg, Germany. The Oxford ____ suggests the name to be a blend of the words gene and chromosome.


# Merge everything into procedures

## Define procedures

In [10]:
######################### NLP Functions #########################

def find_inds_of_NE(tags):
    # Get indices of all named entity tags in the tags list
    l = [i for i, t in enumerate(tags) if not t == 'O']
    return(l)

def join_punctuation(seq, characters='.,;?!'):
    # For joining lists of words together with correct
    # punctuation spacings
    characters = set(characters)
    seq = iter(seq)
    current = next(seq)

    for nxt in seq:
        if nxt in characters:
            current += nxt
        else:
            yield current
            current = nxt

    yield current


def tag_paragraph_NER(paragraph,verbose_mode=False):
    # Use AllenNLP to run NER on paragraph
    from allennlp.predictors import Predictor


    # Set up other global variables
    verbose_mode = False


    # Run it on my test sentence
    predictor = Predictor.from_path("/home/davestanley/src/allennlp/ner-model-2018.12.18.tar.gz")
    results = predictor.predict(sentence=paragraph)
    for word, tag in zip(results["words"], results["tags"]):
        if verbose_mode:
            print(f"{word}\t{tag}")

    return results





def get_two_preceeding_sentences(sentences,ind,verbose_mode=False):
    # Returns a subset sentences:
        # sentences[ind] and sentences[ind-1] and sentences[ind-2]
        # only if they exist
    sentences_subset = []

    # Store current and previous sentences
    sentences_subset.append(sentences[ind-2] if ind >= 2 else str(''))
    sentences_subset.append(sentences[ind-1] if ind >= 1 else str(''))
    sentences_subset.append(sentences[ind])

    if verbose_mode:
        print("Current sentence number=" + str(ind))
        print("Sentence i-2 = " + sentences_subset[0])
        print("Sentence i-1 = " + sentences_subset[1])
        print("Sentence i = " + sentences_subset[2])

    return sentences_subset


def extract_blanked_out_sentences(results,verbose_mode = False):
    # From results word/token list, blank out a random word, and then return
    # the sentences containing that blanked out word, plus a few preceding
    # centences for context.
    from nltk.tokenize import sent_tokenize

    words = results['words']
    tags = results['tags']

    l_NE = find_inds_of_NE(tags)
    if verbose_mode:
        print("Indices of named entities:" + str(l_NE))


    # Choose one at random
    import random
    ind = random.choice(l_NE)


    # Back up blanked out word and word type
    removed_word = words[ind]
    removed_word_tag = words[ind]

    # Replace chosen word with blank
    words_new = words.copy()
    blank_token = '____'
    words_new[ind] = blank_token

    # Rebuild the sentence with appropriate punctuation
    #paragraph_new = ' '.join(words_new).replace(" ,", ",").replace(" .", ".")
    paragraph_new = ' '.join(join_punctuation(words_new))

    # Print this sentence along with the previous sentence together
    if verbose_mode: print(paragraph_new)
    if verbose_mode: print("Answer: " + removed_word)



    # Finally, figure out which sentence contains the blank and only present it and the previous two
    # ===

    # First, figure out the index of the sentence containing the blank
    sentences_new = sent_tokenize(paragraph_new)


    curr_word = 0
    i=0
    for sent in sentences_new:
        i=i+1
        curr_word = curr_word + len(sent.split())
        if ind < curr_word:
            break

    ind_sentence_containing_blank = i-1
    if verbose_mode: print(ind_sentence_containing_blank)


    # if verbose_mode:
    #     # This method searches for the token directly - it is slower, but guaranteed to work)
    #     # Never mind, punctuation messes this method up. No time to fix
    #     sentences_new = sent_tokenize(paragraph_new)


    #     i=0
    #     for sent in sentences_new:
    #         if blank_token in sent.split():
    #             break
    #         i=i+1
    #     ind_sentence_containing_blank2 = i
    #     if verbose_mode: print(ind_sentence_containing_blank2)
    #     if not ind_sentence_containing_blank2 == ind_sentence_containing_blank: print('Error occurred')




    sentences_subset = get_two_preceeding_sentences(sentences_new,ind_sentence_containing_blank)
    paragraph_subset = ' '.join(join_punctuation([sent for sent in sentences_subset if sent]))  # Joins only if sentence is non-empty

    text_blanks = dict()
    text_blanks['text'] = paragraph_subset
    text_blanks['removed_word'] = removed_word
    text_blanks['removed_word_tag'] = removed_word_tag

    return text_blanks


In [176]:

# Choose a paragraph
paragraph = arts[15]['paragraphs'][1]['context']
print(paragraph)

# Tag the paragraph
results = tag_paragraph_NER(paragraph)

01/25/2019 03:12:48 - INFO - allennlp.models.archival -   loading archive file /home/davestanley/src/allennlp/ner-model-2018.12.18.tar.gz
01/25/2019 03:12:48 - INFO - allennlp.models.archival -   extracting archive file /home/davestanley/src/allennlp/ner-model-2018.12.18.tar.gz to temp dir /tmp/tmpklva6x7m


The term was created in 1920 by Hans Winkler, professor of botany at the University of Hamburg, Germany. The Oxford Dictionary suggests the name to be a blend of the words gene and chromosome. However, see omics for a more thorough discussion. A few related -ome words already existed—such as biome, rhizome, forming a vocabulary into which genome fits systematically.


01/25/2019 03:12:53 - INFO - allennlp.common.params -   type = default
01/25/2019 03:12:53 - INFO - allennlp.data.vocabulary -   Loading token dictionary from /tmp/tmpklva6x7m/vocabulary.
01/25/2019 03:12:54 - INFO - allennlp.common.from_params -   instantiating class <class 'allennlp.models.model.Model'> from params {'include_start_end_transitions': False, 'regularizer': [['scalar_parameters', {'alpha': 0.1, 'type': 'l2'}]], 'dropout': 0.5, 'label_encoding': 'BIOUL', 'encoder': {'bidirectional': True, 'dropout': 0.5, 'hidden_size': 200, 'input_size': 1202, 'num_layers': 2, 'type': 'lstm'}, 'type': 'crf_tagger', 'text_field_embedder': {'token_characters': {'embedding': {'embedding_dim': 16}, 'encoder': {'conv_layer_activation': 'relu', 'embedding_dim': 16, 'ngram_filter_sizes': [3], 'num_filters': 128, 'type': 'cnn'}, 'type': 'character_encoding'}, 'tokens': {'embedding_dim': 50, 'trainable': True, 'type': 'embedding'}, 'elmo': {'dropout': 0, 'type': 'elmo_token_embedder', 'do_layer_no

In [8]:

# Print the sentences
text_blanks = extract_blanked_out_sentences(results)

blanked_sentence = text_blanks['text']
removed_word = text_blanks['removed_word']
removed_word_tag = text_blanks['removed_word_tag']


print(blanked_sentence)
print('Answer: ' + removed_word)

The term was created in 1920 by Hans Winkler, professor of botany at the University of Hamburg, Germany. The ____ Dictionary suggests the name to be a blend of the words gene and chromosome.
